In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding = 'latin-1').iloc[:, :2].values
data[:5]

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
target=le.fit_transform(data[:,0])
le.classes_

In [ ]:
unique,count = np.unique(target,return_counts=True)
plt.xticks(unique, unique)
plt.bar(unique,count,color=['blue','orange'])

In [ ]:
print('sentence quantity: {} ,train sample: {} ,test sample: {}'.format(len(target),len(target)*0.8,len(target)*0.2))

In [ ]:
from nltk.corpus import stopwords
Stopwords = stopwords.words('english')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
import re
sms=data[:,1]
sms=[re.sub("[^a-zA-Z]"," ", s) for s in sms]
strings=np.char.split(np.char.lower(sms))
words = [[stemmer.stem(word) for word in string if word not in set(Stopwords)] for string in strings ]
sentence = [' '.join(row) for row in words]
sentence[:5]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
cv = CountVectorizer()
tfidf = TfidfTransformer()
X=sentence
y=tf.keras.utils.to_categorical(target,2)
X=cv.fit_transform(X).toarray()
X[0,:]

In [ ]:
X=tfidf.fit_transform(X).toarray()
X[0,:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=12)

In [ ]:
print('train vs test:{} ,{}'.format(X_train.shape[0],X_test.shape[0]))

In [ ]:
model=tf.keras.Sequential([
        tf.keras.layers.Dense(1024, activation='elu',input_dim=X_train.shape[1],kernel_initializer='normal'),
        tf.keras.layers.Dense(256, activation='elu',kernel_initializer='normal'),
        tf.keras.layers.Dense(64, activation='elu',kernel_initializer='normal'),
        tf.keras.layers.Dense(16, activation='elu',kernel_initializer='normal'),
        tf.keras.layers.Dense(units = 2, activation='softmax')
    ])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=10,batch_size=60)

In [ ]:
y_pred=model.predict(X_test)
Y_pred=np.argmax(y_pred,axis=1)
Y_test=np.argmax(y_test,axis=1)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
ac=accuracy_score(Y_pred,Y_test)
cm=confusion_matrix(Y_pred,Y_test)

In [ ]:
ac

In [ ]:
cm